In [1]:
%matplotlib inline

from modsim import *

In [6]:
# Import relevant units

m = UNITS.meter
kg = UNITS.kilogram
s = UNITS.second
N = UNITS.newton
cm = UNITS.centimeter
g = UNITS.gram
mm = UNITS.millimeter

In [9]:
# Set initial conditions.  These conditions will be used in make_system to generate a system.

condition = Condition(y=0*m, 
                      grav=9.8*m/s**2, 
                      diameter=4*cm, 
                      rho_air=1.2*kg/m**3, 
                      C_d=0.75, 
                      velocity=0*m/s, 
                      duration=100*s, 
                      A_surf=4.5234*cm**2,
                      m_engine=50.6360*g,
                      m_rocket=50*g,
                      rho_fuel=1.599*g/cm**3,
                      B_rate=1*mm/s)

In [ ]:
# Takes in an initial condition and generates a full system

def make_system(condition):
    unpack(condition)
    vy = velocity
    init = State(y=y,vy=vy)
    area_rocket = np.pi * 

In [2]:


def slope_func(state,t,system):
    m, y, vy = state
    unpack(system)
    m_dot = rho * A_surf * B_rate
    ay = (v_exhaust * m_dot) / m - g - F_drag / m
    return vy, ay